In [1]:
import copy

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils import data
import tensorflow as tf
from tqdm import tqdm,trange
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report
from transformers import *

In [2]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
import warnings
import pickle
import random

warnings.filterwarnings('ignore')

data = pd.read_csv("train_ex.txt", sep='\s',skip_blank_lines=True,encoding = 'utf-8')

dev_data = pd.read_csv("sample2.data",sep = '\s',encoding = 'utf-8')
data

,0,1,Sentence#
0,醫,O,1
1,師,O,1
2,：,O,1
3,啊,O,1
4,回,O,1
...,...,...,...
418247,照,O,9406
418248,個,O,9406
418249,X,O,9406
418250,光,O,9406


In [3]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [4]:
!nvidia-smi

Thu Dec 24 10:37:24 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.89       Driver Version: 460.89       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 3090   WDDM  | 00000000:65:00.0  On |                  N/A |
| 67%   63C    P2   304W / 350W |  10081MiB / 24576MiB |     79%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["0"].values.tolist(),
                                                     s["1"].values.tolist())]
        self.grouped = self.data.groupby("Sentence#").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped[self.n_sent]
            self.n_sent += 1
            return s
        except:
            return None

In [6]:
getter = SentenceGetter(data)
dev_getter = SentenceGetter(dev_data)

In [7]:
getter.sentences

[[('醫', 'O'),
  ('師', 'O'),
  ('：', 'O'),
  ('啊', 'O'),
  ('回', 'O'),
  ('去', 'O'),
  ('還', 'O'),
  ('好', 'O'),
  ('嗎', 'O'),
  ('？', 'O'),
  ('民', 'O'),
  ('眾', 'O'),
  ('：', 'O'),
  ('欸', 'O'),
  ('，', 'O'),
  ('還', 'O'),
  ('是', 'O'),
  ('虛', 'O'),
  ('虛', 'O'),
  ('的', 'O'),
  ('，', 'O'),
  ('但', 'O'),
  ('。', 'O'),
  ('醫', 'O'),
  ('師', 'O'),
  ('：', 'O'),
  ('欸', 'O'),
  ('，', 'O'),
  ('真', 'O'),
  ('的', 'O'),
  ('。', 'O')],
 [('民', 'O'),
  ('眾', 'O'),
  ('：', 'O'),
  ('好', 'O'),
  ('險', 'O'),
  ('好', 'O'),
  ('險', 'O'),
  ('。', 'O'),
  ('坦', 'O'),
  ('白', 'O'),
  ('講', 'O'),
  ('我', 'O'),
  ('剛', 'O'),
  ('回', 'O'),
  ('去', 'O'),
  ('的', 'O'),
  ('時', 'O'),
  ('候', 'O'),
  ('晚', 'O'),
  ('上', 'O'),
  ('還', 'O'),
  ('是', 'O'),
  ('有', 'O'),
  ('盜', 'O'),
  ('汗', 'O'),
  ('。', 'O'),
  ('醫', 'O'),
  ('師', 'O'),
  ('：', 'O'),
  ('盜', 'O'),
  ('汗', 'O'),
  ('。', 'O')],
 [('民', 'O'),
  ('眾', 'O'),
  ('：', 'O'),
  ('阿', 'O'),
  ('只', 'O'),
  ('是', 'O'),
  ('前', 'B-time'),
  ('天', 'I-ti

In [8]:
sentences = [[word[0] for word in sentence] for sentence in getter.sentences]
dev_sentences = [[word[0] for word in sentence] for sentence in dev_getter.sentences]
sentences

[['醫',
  '師',
  '：',
  '啊',
  '回',
  '去',
  '還',
  '好',
  '嗎',
  '？',
  '民',
  '眾',
  '：',
  '欸',
  '，',
  '還',
  '是',
  '虛',
  '虛',
  '的',
  '，',
  '但',
  '。',
  '醫',
  '師',
  '：',
  '欸',
  '，',
  '真',
  '的',
  '。'],
 ['民',
  '眾',
  '：',
  '好',
  '險',
  '好',
  '險',
  '。',
  '坦',
  '白',
  '講',
  '我',
  '剛',
  '回',
  '去',
  '的',
  '時',
  '候',
  '晚',
  '上',
  '還',
  '是',
  '有',
  '盜',
  '汗',
  '。',
  '醫',
  '師',
  '：',
  '盜',
  '汗',
  '。'],
 ['民',
  '眾',
  '：',
  '阿',
  '只',
  '是',
  '前',
  '天',
  '好',
  '很',
  '多',
  '。',
  '前',
  '天',
  '就',
  '算',
  '沒',
  '盜',
  '，',
  '可',
  '是',
  '一',
  '覺',
  '到',
  '天',
  '明',
  '這',
  '樣',
  '。',
  '醫',
  '師',
  '：',
  '一',
  '覺',
  '到',
  '天',
  '明',
  '齁',
  '。'],
 ['我',
  '給',
  '你',
  '看',
  '電',
  '腦',
  '斷',
  '層',
  '齁',
  '。',
  '民',
  '眾',
  '：',
  '嘿',
  '。',
  '還',
  '有',
  '那',
  '個',
  '病',
  '毒',
  '報',
  '告',
  '不',
  '知',
  '道',
  '出',
  '來',
  '沒',
  '。',
  '醫',
  '師',
  '：',
  '病',
  '毒',
  '齁',
  '。'],
 ['民',
  '眾',
  '：',
  

In [9]:
labels = [[s[1] for s in sent] for sent in getter.sentences]
dev_labels = [[s[1] for s in sent] for sent in dev_getter.sentences]

In [10]:
tag_values = list(set(data["1"].values))
tag_values.append("PAD")
tag_values
# tag2idx = {t: i for i, t in enumerate(tag_values)}
# tag2idx

['I-profession',
 'O',
 'B-clinical_event',
 'B-time',
 'I-name',
 'I-money',
 'B-money',
 'I-contact',
 'I-family',
 'I-clinical_event',
 'I-ID',
 'B-education',
 'I-education',
 'B-location',
 'B-med_exam',
 'B-profession',
 'I-med_exam',
 'B-contact',
 'I-others',
 'I-location',
 'B-organization',
 'B-ID',
 'I-time',
 'B-name',
 'I-organization',
 'B-family',
 'B-others',
 'PAD']

In [11]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

from keras.preprocessing.sequence import pad_sequences

In [12]:
MAX_LEN = 500
bs = 8

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
n_gpu

1

In [14]:
torch.cuda.get_device_name(0)

'GeForce RTX 3090'

In [15]:
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

In [16]:
print(sentences[0])
a = [tokenizer.tokenize(i) for i in sentences[0]]
print(a)
b = [tokenizer.convert_tokens_to_ids(i) for i in a] 
print(b)

['醫', '師', '：', '啊', '回', '去', '還', '好', '嗎', '？', '民', '眾', '：', '欸', '，', '還', '是', '虛', '虛', '的', '，', '但', '。', '醫', '師', '：', '欸', '，', '真', '的', '。']
[['醫'], ['師'], ['：'], ['啊'], ['回'], ['去'], ['還'], ['好'], ['嗎'], ['？'], ['民'], ['眾'], ['：'], ['欸'], ['，'], ['還'], ['是'], ['虛'], ['虛'], ['的'], ['，'], ['但'], ['。'], ['醫'], ['師'], ['：'], ['欸'], ['，'], ['真'], ['的'], ['。']]
[[7015], [2374], [8038], [1557], [1726], [1343], [6917], [1962], [1621], [8043], [3696], [4707], [8038], [3618], [8024], [6917], [3221], [5995], [5995], [4638], [8024], [852], [511], [7015], [2374], [8038], [3618], [8024], [4696], [4638], [511]]


In [17]:
def encode_sent_labels(sentence,label):
    return tokenizer.encode(sentence),['O']+label+['O']

In [18]:
tokenized_texts_and_labels = [
    encode_sent_labels(sent, labs)
    for sent, labs in zip(sentences, labels)
]
print('done')
dev_tokenized_texts_and_labels = [
    encode_sent_labels(sent, labs)
    for sent, labs in zip(dev_sentences, dev_labels)
]

Token indices sequence length is longer than the specified maximum sequence length for this model (543 > 512). Running this sequence through the model will result in indexing errors


done


In [19]:
tag2idx = {t: i for i, t in enumerate(tag_values)}
tag2idx

{'I-profession': 0,
 'O': 1,
 'B-clinical_event': 2,
 'B-time': 3,
 'I-name': 4,
 'I-money': 5,
 'B-money': 6,
 'I-contact': 7,
 'I-family': 8,
 'I-clinical_event': 9,
 'I-ID': 10,
 'B-education': 11,
 'I-education': 12,
 'B-location': 13,
 'B-med_exam': 14,
 'B-profession': 15,
 'I-med_exam': 16,
 'B-contact': 17,
 'I-others': 18,
 'I-location': 19,
 'B-organization': 20,
 'B-ID': 21,
 'I-time': 22,
 'B-name': 23,
 'I-organization': 24,
 'B-family': 25,
 'B-others': 26,
 'PAD': 27}

In [20]:
tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]
tokenized_labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]

dev_tokenized_texts = [token_label_pair[0] for token_label_pair in dev_tokenized_texts_and_labels]
dev_tokenized_labels = [token_label_pair[1] for token_label_pair in dev_tokenized_texts_and_labels]

In [21]:
idx = 4
print(dev_tokenized_texts[idx])
print(dev_tokenized_labels[idx])

[101, 7015, 2374, 8038, 872, 3300, 2834, 7768, 749, 7350, 8024, 2792, 809, 872, 100, 5498, 2347, 5195, 3760, 2380, 1333, 749, 5456, 511, 102]
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [22]:
# input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
#                           maxlen=MAX_LEN, dtype="long", value=0.0,
#                           truncating="post", padding="post")

# dev_input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in dev_tokenized_texts],
#                           maxlen=MAX_LEN, dtype="long", value=0.0,
#                           truncating="post", padding="post")

In [23]:
# tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in tokenized_labels],
#                      maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
#                      dtype="long", truncating="post")
# dev_tags =pad_sequences([[tag2idx.get(l) for l in lab] for lab in dev_tokenized_labels],
#                      maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
#                      dtype="long", truncating="post")

In [24]:
# attention_masks = [[float(i != 0.0) for i in ii] for ii in input_ids]
# dev_attention_masks = [[float(i != 0.0) for i in ii] for ii in dev_input_ids]

In [25]:
tr_inputs = torch.LongTensor(input_ids)
val_inputs = torch.LongTensor(dev_input_ids)
tr_tags = torch.LongTensor(tags)
val_tags = torch.LongTensor(dev_tags)
tr_masks = torch.tensor(attention_masks)
val_masks = torch.tensor(dev_attention_masks)

NameError: name 'input_ids' is not defined

In [ ]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
# train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data,batch_size=bs,shuffle=True)

# valid_data = TensorDataset(val_inputs,val_masks,val_tags)
# valid_sampler = SequentialSampler(valid_data)
# valid_dataloader = DataLoader(valid_data,batch_size=bs)

In [26]:
model = BertForTokenClassification.from_pretrained("bert-base-chinese",
                                                    num_labels=len(tag_values),
                                                    output_attentions = False,
                                                    output_hidden_states = False)
model.cuda()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [27]:
tokenizer.tokenize

<bound method PreTrainedTokenizer.tokenize of <transformers.tokenization_bert.BertTokenizer object at 0x0000023DC70FE880>>

In [ ]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters())
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

optimizer = AdamW(
    optimizer_grouped_parameters,
    lr=3e-5,
    eps=1e-8
)

In [ ]:
from transformers import get_linear_schedule_with_warmup

epochs = 10
max_grad_norm = 3.0

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)
criteria = torch.nn.CrossEntropyLoss(ignore_index = tag2idx['PAD'])

In [ ]:
## Store the average loss after each epoch so we can plot them.
loss_values, validation_loss_values = [], []

for _ in trange(epochs, desc="Epoch"):
    model.train()
    total_loss = 0

    # Training loop
    predictions , true_labels = [], []
    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)

        b_input_ids, b_input_mask, b_labels = batch
        model.zero_grad()
        outputs = model(b_input_ids, token_type_ids=None,
                        attention_mask=b_input_mask, labels=b_labels)
        loss = outputs[0]
        loss.backward()
        # track train loss
        total_loss += loss.item()
        # Clip the norm of the gradient
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        optimizer.step()
        scheduler.step()
        
        logits = output.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.extend(label_ids)
    avg_train_loss = total_loss / len(train_dataloader)
    print("Average train loss: {}".format(avg_train_loss))
    pred_tags = [tag_values[p_i] for p, l in zip(predictions, true_labels)
                                 for p_i, l_i in zip(p, l) if tag_values[l_i] != "PAD"]
    valid_tags = [tag_values[l_i] for l in true_labels
                                  for l_i in l if tag_values[l_i] != "PAD"]
    print("Training F1-Score: {}".format(f1_score([valid_tags], [pred_tags])))
    loss_values.append(avg_train_loss)


#     model.eval()
#     eval_loss, eval_accuracy = 0, 0
#     predictions , true_labels = [], []
#     for batch in valid_dataloader:
#         batch = tuple(t.to(device) for t in batch)
#         b_input_ids, b_input_mask, b_labels = batch
#         with torch.no_grad():
#             outputs = model(b_input_ids, token_type_ids=None,
#                             attention_mask=b_input_mask, labels=b_labels)
#         logits = outputs[1].detach().cpu().numpy()
#         label_ids = b_labels.to('cpu').numpy()
#         eval_loss += outputs[0].mean().item()
#         predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
#         true_labels.extend(label_ids)

#     eval_loss = eval_loss / len(valid_dataloader)
#     validation_loss_values.append(eval_loss)
#     print("Validation loss: {}".format(eval_loss))
#     pred_tags = [tag_values[p_i] for p, l in zip(predictions, true_labels)
#                                  for p_i, l_i in zip(p, l) if tag_values[l_i] != "PAD"]
#     valid_tags = [tag_values[l_i] for l in true_labels
#                                   for l_i in l if tag_values[l_i] != "PAD"]
#     print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags)))
#     print()
#     print(classification_report(valid_tags,pred_tags ))

### TEst

In [ ]:
test_data = pd.read_csv("test_v2_p.txt",sep = '\s',encoding='utf-8')
test_data

In [ ]:
class testGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [w for w in s["0"].values.tolist()]
        self.grouped = self.data.groupby("Sentence#").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped[self.n_sent]
            self.n_sent += 1
            return s
        except:
            return None

In [ ]:
test_getter = testGetter(test_data)

In [ ]:
# test_sentences = [[word for word in sentence] for sentence in test_getter.sentences]
test_sentences = []
for idx,sentence in enumerate(test_getter.sentences , start = 1):
    _ = []
    c = 0
    for word in sentence:
        _.append(word)
        if len(sentence)>511:
            c += 1
        if c >457:
            test_sentences.append(_)
            _ = []
            c = 0
    test_sentences.append(_)

In [ ]:
a = ['醫', '師', '：', '因', '為', '你', '之', '前', '打', '針', '，', '假', '如', '效', '果', '有', '效', '有', '時', '候', '2', '0', '1', '1', '3', '月', '4', '號', '就', '見', '效', '了', '。']
# a = inpec[2][0]
print(a,len(a))
b = tokenizer.encode(a)
# print(b)
test_i = torch.tensor([b]).cuda()
# print(test_i)
test_o = model(test_i)
# print(test_o[0].shape)
# print(test_o[0].to('cpu').data.numpy())
test_l = np.argmax(test_o[0].to('cpu').data.numpy(), axis=2)
# print(test_l)
test_tag = [tag_values[i] for i in test_l[0][1:-1]]
print(test_tag ,len(test_tag))
# tag_values[label_idx]
# c = tokenizer.convert_ids_to_tokens(b)
# print(c)
# tokenizer.convert_tokens_to_ids('[UNK]')

In [ ]:
pred_valid = []
c = 1
for i in test_sentences:
    if len(i) > 500:
        print(c,len(i))
    try:
        tokenized_sentence = tokenizer.encode(i)
    except:
        print(i)
    input_ids = torch.tensor([tokenized_sentence]).cuda()
    pred_valid.append(input_ids)
    c += 1

In [ ]:
all_label_indices = []
c = 0
for i in pred_valid:
    with torch.no_grad():
        try:
            output = model(i)
        except:
            print(len(i[0]))
            print(c)
    c += 1
    label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)
    all_label_indices.append(label_indices)
all_label_indices

In [ ]:
all_new_tokens = []
all_new_labels = []
for i in range(len(pred_valid)):
    tokens = tokenizer.convert_ids_to_tokens(pred_valid[i].to('cpu').numpy()[0])
    new_tokens, new_labels = [], []
    for token, label_idx in zip(tokens, all_label_indices[i][0]):
        if token.startswith("##"):
            print(token)
            new_tokens[-1] = new_tokens[-1] + token[2:]
        else:
            new_labels.append(tag_values[label_idx])
            new_tokens.append(token)
    all_new_tokens.append(new_tokens)
    all_new_labels.extend(new_labels[1:-1])

In [ ]:
for i,j in zip(all_new_labels,test_data.index):
    test_data['1'][j] = i

In [ ]:
class Upload_f(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t,p) for w, t ,p in zip(s["0"].values.tolist(),
                                                     s["1"].values.tolist(),
                                                     s["2"].values.tolist())]
        self.grouped = self.data.groupby("2").apply(agg_func)
        self.sentences = [s for s in self.grouped]

In [ ]:
getter = Upload_f(test_data)
len(getter.sentences)

In [ ]:
def art_append(art_id,s_id,e_id,text,ner_type):
    q = []
    q.append(art_id)
    q.append(s_id)
    q.append(e_id)
    q.append(text)
    q.append(ner_type)
    return q

upload = []
for sentence in getter.sentences:
    str_len = 0
    res = False
    s = ''
    n_t = ''
    res2 = False
    for sent in sentence:
        if not (res or res2) and (sent[1] == 'O' or 'I-' in sent[1]):
            str_len += len(sent[0])
            continue
        elif not res and 'B-' in sent[1]:
            res = True
            res2 = False
            s = sent[0]
            n_t = sent[1].split('B-')[1]
            st_id = str_len
            str_len += len(sent[0])
            continue
        elif (res or res2) and 'B-' in sent[1]:
            res = True
            if res2:
                res2 = False
            end_id = str_len
            _1 = art_append(sent[2],st_id,end_id,s,n_t)
            upload.append(_1) 
            s = sent[0]
            n_t = sent[1].split('B-')[1]
            st_id = str_len
            str_len += len(sent[0])
            continue
        elif res and ('I-' in sent[1]):
            res2 = True
            s += sent[0]
            str_len += len(sent[0])
            continue
        elif res and (sent[1] == 'O'):
            res = False
            res2 = False
            end_id = str_len
            _1 = art_append(sent[2],st_id,end_id,s,n_t)
            upload.append(_1) 
            s = ''
            n_t = ''
            str_len += len(sent[0])
            continue
    if res2:
        res2 = False
        end_id = str_len
        _1 = art_append(sent[2],st_id,end_id,s,n_t)
        upload.append(_1) 

In [ ]:
with open("./1209_bert_1.tsv","w+",encoding="utf-8") as f: 
    f.write('article_id')
    f.write('\t')
    f.write('start_position')
    f.write('\t')
    f.write('end_position')
    f.write('\t')
    f.write('entity_text')
    f.write('\t')
    f.write('entity_type')
    f.write('\n')
    for q in upload:
        for j in q[:-1]:
            f.write(str(j))
            f.write('\t')
        f.write(str(q[-1]))
        f.write('\n')